In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [2]:
from tqdm.notebook import tqdm
from sklearn.metrics import roc_auc_score, average_precision_score, f1_score, confusion_matrix, balanced_accuracy_score

In [3]:
import os
import random
import gc

In [5]:
# seed=42
# reproducible = True
# # Setting the seed before gin parsing
# os.environ['PYTHONHASHSEED'] = str(seed)
# random.seed(seed)
# np.random.seed(seed)
# torch.manual_seed(seed)
# torch.cuda.manual_seed_all(seed)  # For multi-GPU setups

# if reproducible:
#     os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8'
#     torch.use_deterministic_algorithms(True)
#     torch.backends.cudnn.deterministic = True
#     torch.backends.cudnn.benchmark = False

# get data

In [6]:
class TorchDataset(Dataset):
    def __init__(self, data_path, labels_path):
        self.data = torch.load(data_path)
        self.labels = torch.load(labels_path)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        data = self.data[idx]
        labels = self.labels[idx]
        # return torch.tensor(data, dtype=torch.float32), torch.tensor(labels, dtype=torch.float32)
        return data.clone().detach(), labels.clone().detach()

In [7]:
# Example usage for creating datasets
train_dataset = TorchDataset('Phenotyping/Ptrain.pt', 'Phenotyping/y_train.pt')
valid_dataset = TorchDataset('Phenotyping/Pvalid.pt', 'Phenotyping/y_valid.pt')
test_dataset  = TorchDataset('Phenotyping/Ptest.pt', 'Phenotyping/y_test.pt')

In [8]:
train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True, num_workers=1, pin_memory=True, prefetch_factor=2)
valid_loader = DataLoader(valid_dataset, batch_size=256, shuffle=False, num_workers=1, pin_memory=True, prefetch_factor=2)
test_loader  = DataLoader(test_dataset,  batch_size=256, shuffle=False, num_workers=1, pin_memory=True, prefetch_factor=2)

## model as in Gandin, Ilaria, et al. "Interpretability of time-series deep learning models: A study in cardiovascular patients admitted to Intensive care unit." Journal of biomedical informatics 121 (2021): 103876.

In [9]:
import torch
import torch.nn as nn

# Define the Attention Layer
class Attention(nn.Module):
    def __init__(self, shape):
        super(Attention, self).__init__()
        self.attention_dense = nn.Linear(shape, shape)
    
    def forward(self, inputs):
        # Apply linear transformation
        a = self.attention_dense(inputs)
        # Apply softmax to get attention scores
        attention_scores = torch.softmax(a, dim=-1)
        # Element-wise multiplication with inputs
        output_attention_mul = inputs * attention_scores
        return output_attention_mul, attention_scores

# Define the Model
class CustomModel_attext(nn.Module):
    def __init__(self, n_wind, n_features, dense_nparams1=256, n_classes=15):  # n_classes is added
        super(CustomModel_attext, self).__init__()
        self.attention = Attention(n_features)
        self.lstm = nn.LSTM(n_features, dense_nparams1, num_layers=1, batch_first=True)
        self.dense = nn.Linear(dense_nparams1, n_classes)  # Output layer now has n_classes outputs
    
    def forward(self, x, return_attention=False):
        # Get the attended output and attention scores
        x, attention_scores = self.attention(x)
        # Pass through LSTM
        x, _ = self.lstm(x)
        # Extract the last output of the LSTM
        x = x[:, -1, :]
        # Apply the dense layer to get logits (no softmax)
        logits = self.dense(x)
        
        if return_attention:
            return logits, attention_scores
        return logits

In [11]:
epochs = 100
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [12]:
# Create the model
model = CustomModel_attext(288, 231, 15)
criterion = criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=5e-4, weight_decay=1e-6)

In [13]:

def train(train_loader, model, criterion, optimizer, device):
    output_list = []
    target_list = [] 
    # Switch to train mode
    model = model.to(device)
    model.train()

    for count, (data, label) in enumerate(train_loader):
        # Move data and labels to the specified device (GPU/CPU)
        data = data.to(device).type(torch.float)
        label = label.to(device).type(torch.long)  # Ensure labels are long type

        # Debugging prints
        # print(f"Batch {count} - Label unique values: {torch.unique(label)}")

        # Compute output (logits from the model)
        output = model(data)

        # Compute the loss
        loss = criterion(output, label)

        # Convert output and target to lists for tracking
        output_np = output.argmax(dim=1).detach().cpu().numpy().tolist()  # Get predicted class indices
        target_np = label.detach().cpu().numpy().tolist()
        output_list += output_np
        target_list += target_np

        # Compute gradient and do an optimizer step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    return output_list, target_list


def validate(val_loader, model, criterion, device):
    output_list = []
    target_list = [] 
    # Switch to evaluate mode
    model.eval()

    with torch.no_grad():
        for count, (data, label) in enumerate(val_loader):
            # Move data and labels to the specified device (GPU/CPU)
            data = data.to(device).type(torch.float)
            label = label.to(device).type(torch.long)  # Ensure labels are long type

            # Debugging prints
            # print(f"Batch {count} - Label unique values: {torch.unique(label)}")

            # Compute output (logits from the model)
            output = model(data)

            # Compute the loss
            loss = criterion(output, label)

            # Convert output and target to lists for tracking
            output_np = output.argmax(dim=1).detach().cpu().numpy().tolist()  # Get predicted class indices
            target_np = label.detach().cpu().numpy().tolist()
            output_list += output_np
            target_list += target_np

    return output_list, target_list

In [14]:
# Assuming you have a variable `seed` already defined
model_save_dir = './model_checkpoints/'  # Directory to save models

# Create the directory if it doesn't exist
os.makedirs(model_save_dir, exist_ok=True)

In [ ]:
for i in tqdm(range(1000)):
    
    seed = i
    reproducible = True
    # Setting the seed before gin parsing
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # For multi-GPU setups

    if reproducible:
        os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8'
        torch.use_deterministic_algorithms(True)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
    
    epoch_list = [] 
    bal_acc_train = []
    bal_acc_val = []
    best_bal_acc = 0

    model = CustomModel_attext(288, 231, 15)
    criterion = criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=5e-4, weight_decay=1e-6)
    
    print(f'Starting training for seed {seed}')

    for epoch in range(epochs):
        epoch_list.append(epoch)

        # Train for one epoch
        output_train_per, target_train_per = train(train_loader, model, criterion, optimizer, device)

        # Calculate balanced accuracy for training data
        balanced_acc_train = balanced_accuracy_score(target_train_per, output_train_per)
        bal_acc_train += [balanced_acc_train]


        # Evaluate on validation set
        output_val_per, target_val_per = validate(test_loader, model, criterion, device)

        # Calculate balanced accuracy for validation data
        balanced_acc_val = balanced_accuracy_score(target_val_per, output_val_per)
        bal_acc_val += [balanced_acc_val]

        # Update the best balanced accuracy
        if balanced_acc_val > best_bal_acc:
            best_bal_acc = balanced_acc_val
            torch.save(model.state_dict(), os.path.join(model_save_dir, f'bm_bal_acc_seed_{seed:03d}.pth'))

            
    print(f'Completed training and model saving for seed {seed}')

    del model
    # collect cache
    gc.collect()
    torch.cuda.empty_cache()